In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

### In this notebook we will use a pre-trained model to classify images, to see if it's better than building your own model from scratch.

##### We will use MobileNet which is a deep learning model architecture that is designed for efficient and lightweight image classification tasks.

In [2]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.layers import Flatten, Dense
from skimage.transform import resize
from keras.layers import Dropout

# Load the MobileNet model pre-trained on ImageNet data
base_model = MobileNet(
    weights='imagenet', include_top=False, input_shape=(96, 96, 3))
x = base_model.output
x = Flatten()(x)  # Flatten the output layer to one dimension
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])


# Function to preprocess the dataset
def preprocess_data(X, target_size):
    # Reshape and resize the data
    X_reshaped = np.reshape(X.values, (-1, 48, 48))
    X_resized = np.array([resize(img, (target_size, target_size))
                         for img in X_reshaped])
    X_rgb = np.repeat(X_resized[..., np.newaxis], 3, axis=3)

    # Preprocess the input for MobileNet
    X_preprocessed = preprocess_input(X_rgb)
    return X_preprocessed


# Load dataset
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")["0"]

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")["0"]

# Preprocess the data
X_train_preprocessed = preprocess_data(X_train, 96)
X_test_preprocessed = preprocess_data(X_test, 96)

# Convert y to one-hot encoding
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Train the model
model.fit(X_train_preprocessed, y_train_encoded, epochs=10, batch_size=64)

# Evaluate the model
predictions = model.predict(X_test_preprocessed)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == y_test)

print(f'Accuracy: {accuracy}')

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-29 23:47:39.893710: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-29 23:47:39.893839: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2023-11-29 23:47:50.930161: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-29 23:47:52.198254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


152/152 [==============================] - 26s 128ms/step - loss: 0.4458 - accuracy: 0.9061
Epoch 2/10
152/152 [==============================] - 17s 114ms/step - loss: 0.1212 - accuracy: 0.9763
Epoch 3/10
152/152 [==============================] - 17s 113ms/step - loss: 0.0264 - accuracy: 0.9918
Epoch 4/10
152/152 [==============================] - 14s 92ms/step - loss: 0.0152 - accuracy: 0.9962
Epoch 5/10
152/152 [==============================] - 17s 111ms/step - loss: 0.0294 - accuracy: 0.9933
Epoch 6/10
152/152 [==============================] - 17s 113ms/step - loss: 0.0154 - accuracy: 0.9961
Epoch 7/10
152/152 [==============================] - 17s 113ms/step - loss: 0.0074 - accuracy: 0.9973
Epoch 8/10
152/152 [==============================] - 13s 85ms/step - loss: 0.0044 - accuracy: 0.9990
Epoch 9/10
152/152 [==============================] - 12s 78ms/step - loss: 0.0347 - accuracy: 0.9946
Epoch 10/10
152/152 [==============================] - 12s 77ms/step - loss: 0.0497 - a

2023-11-29 23:50:33.836176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


97/97 [==============================] - 2s 15ms/step
Accuracy: 0.9948220064724919


## Here we got 99.5% accuracy using pretrained model MobileNet which is the highest score which we have ever got

### Using the same model, but with synthetic data

In [4]:
import numpy as np
import pandas as pd
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from keras.layers import Flatten, Dense
from skimage.transform import resize
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

# Data Augmentation setup
datagen = ImageDataGenerator(
    rotation_range=20,      # Rotate images by up to 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2, # Shift images vertically by up to 20% of the height
    shear_range=0.2,        # Shear images by up to 20 degrees
    zoom_range=0.2,         # Zoom in on images by up to 20%
    horizontal_flip=True,   # Allow horizontal flipping
    fill_mode='nearest'     # Fill in newly created pixels
)

# Load the MobileNet model pre-trained on ImageNet data
base_model = MobileNet(
    weights='imagenet', include_top=False, input_shape=(96, 96, 3))
x = base_model.output
x = Flatten()(x)  # Flatten the output layer to one dimension

predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Function to preprocess the dataset


def preprocess_data(X, target_size):
    # Reshape and resize the data
    X_reshaped = np.reshape(X.values, (-1, 48, 48))
    X_resized = np.array([resize(img, (target_size, target_size))
                         for img in X_reshaped])
    X_rgb = np.repeat(X_resized[..., np.newaxis], 3, axis=3)

    # Preprocess the input for MobileNet
    X_preprocessed = preprocess_input(X_rgb)
    return X_preprocessed


# Load dataset
X_train = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_train_all.csv')

X_test = pd.read_csv(
    '/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/x_test_all.csv')

y_train = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_train_all.csv")["0"]

y_test = pd.read_csv(
    "/Users/ilya/Desktop/Course_work_Data_mining/CompleteDataSet/y_test_all.csv")["0"]

# Preprocess the data
X_train_preprocessed = preprocess_data(X_train, 96)
X_test_preprocessed = preprocess_data(X_test, 96)

# Convert y to one-hot encoding
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Fit the model with data augmentation
model.fit(datagen.flow(X_train_preprocessed, y_train_encoded, batch_size=64),
          steps_per_epoch=len(X_train_preprocessed) / 64, epochs=10)

# Evaluate the model
predictions = model.predict(X_test_preprocessed)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == y_test)

print(f'Accuracy: {accuracy}')

Epoch 1/10


2023-11-30 00:01:46.962649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


151/151 [==============================] - 15s 83ms/step - loss: 0.9143 - accuracy: 0.7939
Epoch 2/10
151/151 [==============================] - 12s 78ms/step - loss: 0.1469 - accuracy: 0.9517
Epoch 3/10
151/151 [==============================] - 12s 77ms/step - loss: 0.1543 - accuracy: 0.9575
Epoch 4/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0900 - accuracy: 0.9702
Epoch 5/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0966 - accuracy: 0.9721
Epoch 6/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0640 - accuracy: 0.9789
Epoch 7/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0670 - accuracy: 0.9815
Epoch 8/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0511 - accuracy: 0.9840
Epoch 9/10
151/151 [==============================] - 12s 78ms/step - loss: 0.0487 - accuracy: 0.9849
Epoch 10/10
151/151 [==============================] - 12s 79ms/step - loss: 0.0442 - accurac

2023-11-30 00:03:47.688235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


97/97 [==============================] - 2s 13ms/step
Accuracy: 0.9715210355987055


### Using synthetic data for pre-trained model just makes it worse. We got only 97.1 % accuracy